In [ ]:
import tiktoken
import pandas as pd


df = pd.read_csv("../data/results/test_all.csv")
encoding = tiktoken.get_encoding("cl100k_base")


input_tokens = []
output_tokens = []

for x in df.to_dict("records"):
    num_input_tokens = len(encoding.encode(x["inputs"]))
    num_output_tokens = len(encoding.encode(x["outputs"]))
    input_tokens.append(num_input_tokens)
    output_tokens.append(num_output_tokens)


input_costs = []
output_costs = []

for tokens in input_tokens:
    input_costs.append(round(tokens * 0.0005 / 1000, 4))

for tokens in output_tokens:
    output_costs.append(round(tokens *  0.0015 / 1000, 4))

print("Avg. input token: ", sum(input_tokens)/len(input_tokens))
print("Avg. output token costs: ", sum(output_tokens)/len(output_tokens))


In [1]:
from cval import CVal
from data import Dependency

CONFIG_FILE = "../config.toml"
ENV_FILE = "../.env"
EVAL_DATA_DIR = "../data/evaluation/data"
INDEX_NAME = "tech-docs"

dep = Dependency(
    project="piggymetrics",
    dependency_category="value-equality",
    option_name="EXPOSE",
    option_value="8080",
    option_type="PORT",
    option_file="Dockerfile",
    option_technology="Docker",
    dependent_option_name="server.port",
    dependent_option_value="8080",
    dependent_option_file="application.yml",
    dependent_option_type="PORT",
    dependent_option_technology="Spring-Boot"
)

cval = CVal.init(
    config_file=CONFIG_FILE,
    env_file=ENV_FILE
)

/home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


[07/10/24 16:20:57] INFO     PyTorch version 2.3.0 available.                                          ]8;id=814274;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/datasets/config.py\config.py]8;;\:]8;id=989730;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/datasets/config.py#58\58]8;;\

[07/10/24 16:20:58] INFO     Ingestion engine initialized.                                   ]8;id=876807;file:///home/simisimon/GitHub/cval/src/ingestion_engine.py\ingestion_engine.py]8;;\:]8;id=772232;file:///home/simisimon/GitHub/cval/src/ingestion_engine.py#45\45]8;;\

                    INFO     Ingestion engine initialized.                                   ]8;id=955623;file:///home/simisimon/GitHub/cval/src/ingestion_engine.py\ingestion_engine.py]8;;\:]8;id=763854;file:///home/simisimon/GitHub/cval/src/ingestion_engine.py#45\45]8;;\

                    INFO     Retrieval engine initialized.                                   ]8;id=622973;file:///home/simisimon/GitHub/cval/src/retrieval_engine.py\retrieval_engine.py]8;;\:]8;id=150357;file:///home/simisimon/GitHub/cval/src/retrieval_engine.py#29\29]8;;\

                    INFO     GPT (gpt-4o-2024-05-13) generator initialized.                         ]8;id=948965;file:///home/simisimon/GitHub/cval/src/generator.py\generator.py]8;;\:]8;id=597916;file:///home/simisimon/GitHub/cval/src/generator.py#47\47]8;;\

                    INFO     Initialize CfgNet prompt settings.                              ]8;id=854306;file:///home/simisimon/GitHub/cval/src/prompt_settings.py\prompt_settings.py]8;;\:]8;id=421088;file:///home/simisimon/GitHub/cval/src/prompt_settings.py#192\192]8;;\

                    INFO     CVal initialized.                                                           ]8;id=577499;file:///home/simisimon/GitHub/cval/src/cval.py\cval.py]8;;\:]8;id=73709;file:///home/simisimon/GitHub/cval/src/cval.py#63\63]8;;\

In [2]:
response = cval.query(dependency=dep, index_name="so-posts")

[07/10/24 16:21:01] INFO     Query LLM with context.                                                    ]8;id=952252;file:///home/simisimon/GitHub/cval/src/cval.py\cval.py]8;;\:]8;id=251120;file:///home/simisimon/GitHub/cval/src/cval.py#129\129]8;;\

                    INFO     Rewrite query.                                                             ]8;id=698704;file:///home/simisimon/GitHub/cval/src/cval.py\cval.py]8;;\:]8;id=317105;file:///home/simisimon/GitHub/cval/src/cval.py#136\136]8;;\

                    INFO     Select index: so-posts.                                         ]8;id=116215;file:///home/simisimon/GitHub/cval/src/retrieval_engine.py\retrieval_engine.py]8;;\:]8;id=517583;file:///home/simisimon/GitHub/cval/src/retrieval_engine.py#46\46]8;;\

[07/10/24 16:21:03] INFO     HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200  ]8;id=553114;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=514582;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

[07/10/24 16:21:04] INFO     Create Colbert reranker.                                        ]8;id=273545;file:///home/simisimon/GitHub/cval/src/retrieval_engine.py\retrieval_engine.py]8;;\:]8;id=175763;file:///home/simisimon/GitHub/cval/src/retrieval_engine.py#75\75]8;;\

[07/10/24 16:21:10] INFO     Rerank 10 retrieved nodes into 5 nodes.                        ]8;id=70456;file:///home/simisimon/GitHub/cval/src/retrieval_engine.py\retrieval_engine.py]8;;\:]8;id=312645;file:///home/simisimon/GitHub/cval/src/retrieval_engine.py#104\104]8;;\

                    INFO     Query gpt-4o-2024-05-13                                                    ]8;id=892240;file:///home/simisimon/GitHub/cval/src/cval.py\cval.py]8;;\:]8;id=629536;file:///home/simisimon/GitHub/cval/src/cval.py#108\108]8;;\

[07/10/24 16:21:17] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=461721;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=545512;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

In [3]:
response.response

'{\n\t"plan": "1. Identify the purpose of the EXPOSE directive in Dockerfile and the server.port configuration in Spring-Boot\'s application.yml. 2. Determine how these configurations interact within the context of Docker and Spring-Boot. 3. Evaluate if the values of these configurations need to be identical for the system to function correctly. 4. Consider the context provided in the examples to see if there are any explicit mentions of dependencies between these configurations. 5. Conclude whether a value-equality dependency exists based on the gathered information.",\n\t"rationale": "The EXPOSE directive in a Dockerfile is used to inform Docker that the container listens on the specified network ports at runtime. The server.port configuration in Spring-Boot\'s application.yml specifies the port on which the Spring-Boot application will run. For the Docker container to correctly route traffic to the Spring-Boot application, the port exposed by Docker (EXPOSE 8080) must match the port

In [20]:
from llama_index.core import PromptTemplate
import openai

client = openai.OpenAI(base_url="http://172.26.92.115/", api_key="a29859a1-e871-47c5-b15e-57be20c61e8c")


#prompt = PromptTemplate(
#    "What are the key details in the context below about the option Expose from Docker and the option server.port from Spring-Boot to validate if they depend on each other du to value equality?\n"
#    "Context: {context_str}"
#)

prompt = PromptTemplate(
    "Summarize the relevant information related to the option Expose from Docker and the option server.port from Spring-Boot from the context below?"
    "Context: {context_str}"
)

messages = [
    {
        "role": "user",
        "content": prompt.format(
            context_str="\n\n".join([node.get_content() for node in response.source_nodes])
        )
    }
]

answer = client.chat.completions.create(
    messages=messages,
    model="gpt-4o-2024-05-13"
)

[07/08/24 11:13:34] INFO     HTTP Request: POST http://172.26.92.115/chat/completions "HTTP/1.1 200 ]8;id=360784;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=788142;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

In [21]:
answer.choices[0].message.content


'Sure! Here is the summarized information:\n\n**Docker `EXPOSE` Option:**\n- Used in Dockerfiles to specify the port on which a container listens for connections.\n- Does not actually publish the port, but is a form of documentation for others who use the container.\n- To make the port accessible, it requires running the container with the `-p` or `--publish` flag to map the container port to a port on the host machine.\n\n**Spring Boot `server.port` Option:**\n- Configures the HTTP port on which a Spring Boot application listens for incoming connections.\n- Set in the `application.properties` or `application.yml` file.\n- The default port is `8080`, but it can be changed to any available port by setting `server.port=<desired_port>`.\n\nIn summary, `EXPOSE` in Docker is about specifying which port within the container will be used, while `server.port` in Spring Boot tells the application on which port to run. When using both together, you should ensure that the port Spring Boot listens